# Download MRMS data

## Initial configuration

In [1]:
from pandas import Timestamp, Timedelta
import emaremes as mrms

The first step is to configure where the data will be stored locally. By default, the data will be stored in the `emaremes` folder in your home directory.

Setting up these paths is done with the `path_config` object.

In [2]:
path_config = mrms.fetch.path_config
path_config

defaultpath: /home/edwin/emaremes
prefered: /home/edwin/emaremes

To set another directory to store GRIB files, you can use the following command:

In [3]:
new_dir = "./data"
path_config.set_prefered(new_dir)

Prefered path to store *new* Gribfiles is  data


We can add other folders where GRIB files are stored. `emaremes` only downloads the files that are not already present in the folder specified in `path_config`. This is useful to connect other local folders with MRMS data like external drives.

In [4]:
another_dir = "./other_data"
path_config.add_path(another_dir)

print(path_config.prefered_path)

data


To list all the paths available, we can print the `all_paths` attribute of `path_config`:

In [5]:
print(path_config.all_paths)

{PosixPath('/home/edwin/emaremes'), PosixPath('data'), PosixPath('other_data')}


## Download precipitation rate data

MRMS data is available every 2 minutes. We can download the data using the `fetch` module in emaremes.

In [6]:
preciprate_files = mrms.fetch.timerange(
    "2024-09-27T12:00:00",
    "2024-09-28",
    frequency=Timedelta(minutes=60),
    verbose=True,
)

-> 13 files were requested...
-> 13 *new* files will be downloaded...
Saved data/20240928/PrecipRate_00.00_20240928-000000.grib2.gz :)
Saved data/20240927/PrecipRate_00.00_20240927-140000.grib2.gz :)
Saved data/20240927/PrecipRate_00.00_20240927-220000.grib2.gz :)
Saved data/20240927/PrecipRate_00.00_20240927-150000.grib2.gz :)
Saved data/20240927/PrecipRate_00.00_20240927-230000.grib2.gz :)
Saved data/20240927/PrecipRate_00.00_20240927-160000.grib2.gz :)
Saved data/20240927/PrecipRate_00.00_20240927-180000.grib2.gz :)
Saved data/20240927/PrecipRate_00.00_20240927-120000.grib2.gz :)
Saved data/20240927/PrecipRate_00.00_20240927-130000.grib2.gz :)
Saved data/20240927/PrecipRate_00.00_20240927-170000.grib2.gz :)
Saved data/20240927/PrecipRate_00.00_20240927-210000.grib2.gz :)
Saved data/20240927/PrecipRate_00.00_20240927-190000.grib2.gz :)
Saved data/20240927/PrecipRate_00.00_20240927-200000.grib2.gz :)


`fetch.timerange` start and end times are both inclusive:

In [7]:
print("First file: ", preciprate_files[0])
print("Last file: ", preciprate_files[-1])

First file:  data/20240927/PrecipRate_00.00_20240927-120000.grib2.gz
Last file:  data/20240928/PrecipRate_00.00_20240928-000000.grib2.gz


These files are stored locally and new network requests will be skipped if these files are fetched again.

In [8]:
preciprate_files = mrms.fetch.timerange(
    "2024-09-27T12:00:00",
    "2024-09-28",
    frequency=Timedelta(minutes=60),
    verbose=True,
)

-> 13 files were requested...
Nothing new to download :D


## Download other `data_type`

The `fetch.timerange` function downloads precipiration rate data by default. Besides precipiration rate data, other `data_type` are available, like daily accumulated precipitation and precipitation flags. 


### Precipitation flags
Precipitation flags refer to the following classification, derived from the [mrms-support repo](https://github.com/NOAA-National-Severe-Storms-Laboratory/mrms-support/blob/3edf7c25f503f81a12eb179dbbd3d4dae607e477/GRIB2_TABLES/UserTable_MRMS_PrecipFlags.csv): 

|Flag|Description|
|:-:|:-|
|-3|no coverage|
|0|no precipitation|
|1|warm stratiform rain|
|3|snow|
|6|convective rain|
|7|rain mixed with hail|
|10|cold stratiform rain|
|91|tropical/stratiform rain mix|
|96|tropical/convective rain mix|

To download precipitation flags, use the `data_type="precip_flag"` keyword argument:

In [9]:
precipflags_files = mrms.fetch.timerange(
    "2024-09-27T12:00:00",
    Timestamp("2024-09-28T00:00:00"),
    frequency=Timedelta(minutes=60),
    data_type="precip_flag",
    verbose=True,
)

-> 13 files were requested...
-> 13 *new* files will be downloaded...
Saved data/20240927/PrecipFlag_00.00_20240927-120000.grib2.gz :)
Saved data/20240927/PrecipFlag_00.00_20240927-170000.grib2.gz :)
Saved data/20240927/PrecipFlag_00.00_20240927-160000.grib2.gz :)
Saved data/20240927/PrecipFlag_00.00_20240927-140000.grib2.gz :)
Saved data/20240927/PrecipFlag_00.00_20240927-130000.grib2.gz :)
Saved data/20240927/PrecipFlag_00.00_20240927-150000.grib2.gz :)
Saved data/20240927/PrecipFlag_00.00_20240927-220000.grib2.gz :)
Saved data/20240927/PrecipFlag_00.00_20240927-200000.grib2.gz :)
Saved data/20240927/PrecipFlag_00.00_20240927-210000.grib2.gz :)Saved data/20240927/PrecipFlag_00.00_20240927-190000.grib2.gz :)

Saved data/20240928/PrecipFlag_00.00_20240928-000000.grib2.gz :)
Saved data/20240927/PrecipFlag_00.00_20240927-180000.grib2.gz :)Saved data/20240927/PrecipFlag_00.00_20240927-230000.grib2.gz :)



### Download accumulated precipitation

Accumulates are available for 1, 24 and 72 hours periods. To fetch this data, set `data_type` to `"precip_accum_1h"`, `"precip_accum_24h"` or `"precip_accum_72h"`.

In [10]:
daily_files = mrms.fetch.timerange(
    "2024-09-26T12:00:00",
    "2024-09-29",  # <-- If only a date is provided, it defaults to midnight
    frequency=Timedelta(days=1),
    data_type="precip_accum_24h",
)

-> 3 files were requested...


In the last example, we did not specify `verbose=True`, so it only printed the number of files that were downloaded.